In [17]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import os


# Cuda check and visualize data

In [18]:
print("PyTorch CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No device")

PyTorch CUDA version: 12.4
CUDA available: True
CUDA devices: 1
CUDA device name: NVIDIA GeForce GTX 1650


In [19]:
def visualize_image_with_bboxes(image_path, annotation_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with open(annotation_path, 'r') as file:
        annotations = file.readlines()

    height, width, _ = image.shape

    for annotation in annotations:
        values = annotation.strip().split()
        class_id = int(values[0])
        cx, cy, w, h = map(float, values[1:])

        x_min = int((cx - w / 2) * width)
        y_min = int((cy - h / 2) * height)
        x_max = int((cx + w / 2) * width)
        y_max = int((cy + h / 2) * height)

        color = (255, 255, 255)
        if class_id == 0:
            color = (255, 0, 0)
        elif class_id == 1:
            color = (255, 255, 0)

        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 12)

    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

visualize_image_with_bboxes("datasets/cocoa_diseases/images/train/Fito1.jpg", "datasets/cocoa_diseases/labels/train/Fito1.txt")

<Figure size 1000x1000 with 1 Axes>

# Regular YOLO

In [20]:
model = YOLO("yolo11n.pt")

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
print(results)

Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/cocoa_diseases/cocoa_dataset.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=5, project=models/yolo11, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\train.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]
val: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]


Plotting labels to models\yolo11\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 5 dataloader workers
Logging results to models\yolo11\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      1.41G      1.422      3.089      1.282         24        640: 100%|██████████| 63/63 [00:13<00:00,  4.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.40it/s]

                   all         62        289      0.682      0.126      0.252      0.165


...


100 epochs completed in 0.475 hours.
Optimizer stripped from models\yolo11\train\weights\last.pt, 5.5MB
Optimizer stripped from models\yolo11\train\weights\best.pt, 5.5MB

Validating models\yolo11\train\weights\best.pt...
Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.81it/s]


                   all         62        289      0.663      0.718      0.744      0.489
          phytophthora         30         39      0.692      0.634        0.7      0.488
               monilia         23         30      0.608      0.724      0.703        0.5
               healthy         52        220      0.689      0.795      0.829      0.479
Speed: 0.5ms preprocess, 8.8ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to models\yolo11\train
Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:09<00:00,  1.62it/s]


                   all         62        289      0.663      0.718      0.744      0.486
          phytophthora         30         39      0.692      0.634      0.701      0.483
               monilia         23         30      0.608      0.724      0.701      0.494
               healthy         52        220      0.689      0.795      0.829      0.479
Speed: 0.7ms preprocess, 11.8ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to models\yolo11\train2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000194840A3BD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012

# Pruning setup 

In [21]:
def prune_model(pytorch_model, amount):
    for name, module in pytorch_model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=amount)
            prune.remove(module, "weight")
    return pytorch_model

# Pre-train pruning YOLO

## No retrain 

In [ ]:
model = YOLO("yolo11n.pt")
model.model = prune_model(model.model, 0.05)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11_pruned_no_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
print(results)

Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/cocoa_diseases/cocoa_dataset.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=5, project=models/yolo11_pruned_no_retrain, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

train: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\train.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


## Retrain on coco

In [ ]:
model = YOLO("yolo11n.pt")
model.model = prune_model(model.model, 0.05)

# Retrain on coco8 before training on cocoa dataset
model.train(
    data="datasets/coco8/coco8_dataset.yaml",
    epochs=100,                     
    imgsz=640,
    batch=4,
    device="cuda",
    workers=5             
)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11_pruned_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
print(results)

# Post-train pruning YOLO

## Post prune retrain

In [ ]:
# Get last train x best model
newest_train_dir = max(
    (os.path.join("models/yolo11/", d) for d in os.listdir("models/yolo11/") if d.startswith("train")),
    key=os.path.getmtime
)
weights_dir = os.path.join(newest_train_dir, "weights")
best_model_path = os.path.join(weights_dir, "best.pt")

model = YOLO(best_model_path)
model.model = prune_model(model.model, 0.05)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11_post_pruned_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
print(results)

## Post prune 

In [ ]:
newest_train_dir = max(
    (os.path.join("models/yolo11/", d) for d in os.listdir("models/yolo11/") if d.startswith("train")),
    key=os.path.getmtime
)
weights_dir = os.path.join(newest_train_dir, "weights")
best_model_path = os.path.join(weights_dir, "best.pt")

model = YOLO(best_model_path)
model.model = prune_model(model.model, 0.05)

results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
print(results)